In [ ]:
import re

# === Análisis léxico ===
token_patron = {
    "KEYWORD": r'\b(if|else|for|while|print|int|float|void|return|function|def|class|static)\b',
    "IDENTIFIER": r'\b[a-zA-Z_][a-zA-Z0-9_]*\b',
    "NUMBER": r'\b\d+(\.\d+)?\b',
    "ARITHMETIC_OPERATOR": r'[+\-*/^%]|\/\/',  # ampliamos los operadores aritméticos
    "ASSIGNMENT_OPERATOR": r'=|\+=|-=|\*=|\/=|%=',  # últimos operadores
    "COMPARISON_OPERATOR": r'[<>]=?|==|!=',
    "LOGICAL_OPERATOR": r'\band\b|\bor\b|\bnot\b',  # operadores lógicos
    "DELIMITER": r'[().;{},\[\]]',
    "WHITESPACE": r'\s+',
    "STRING": r'"[^"]*"',
    "COMMENT": r'\/\/.*|\/\*[\s\S]*?\*\/'
}

class Token:
    def __init__(self, tipo, valor, posicion):
        self.tipo = tipo
        self.valor = valor
        self.posicion = posicion
    
    def __str__(self):
        return f'Token({self.tipo}, {self.valor}, pos={self.posicion})'

def identificar_tokens(texto):
    patron_general = '|'.join(f'(?P<{token}>{patron})' for token, patron in token_patron.items())
    patron_regex = re.compile(patron_general)
    
    tokens_encontrados = []
    posicion = 0
    
    for match in patron_regex.finditer(texto):
        tipo = match.lastgroup
        valor = match.group()
        if tipo not in ["WHITESPACE", "COMMENT"]:
            tokens_encontrados.append(Token(tipo, valor, posicion))
        posicion = match.end()
    
    return tokens_encontrados

# === Análisis sintáctico ===
class Parser:
    def __init__(self, tokens):
        self.tokens = tokens
        self.pos = 0
        self.funciones = {}
    
    def obtener_token_actual(self):
        return self.tokens[self.pos] if self.pos < len(self.tokens) else None
    
    def avanzar(self):
        self.pos += 1
        return self.obtener_token_actual()
    
    def coincidir(self, tipo_esperado):
        token = self.obtener_token_actual()
        if token and token.tipo == tipo_esperado:
            return self.avanzar()
        raise SyntaxError(f'Error sintáctico: se esperaba {tipo_esperado}, pero se encontró: {token}')
    
    def parse_funcion(self):
        """Analiza la definición de una función"""
        token = self.obtener_token_actual()
        if token.valor in ['function', 'def']:
            self.avanzar()
            nombre = self.coincidir("IDENTIFIER")
            self.coincidir("DELIMITER")  
            parametros = self.parse_parametros()
            self.coincidir("DELIMITER")
            self.coincidir("DELIMITER")
            cuerpo = self.parse_bloque()
            self.coincidir("DELIMITER")
            
            self.funciones[nombre.valor] = {
                'parametros': parametros,
                'cuerpo': cuerpo
            }
    
    def parse_parametros(self):
        """Analiza los parámetros de una función"""
        parametros = []
        while True:
            token = self.obtener_token_actual()
            if token.tipo == "DELIMITER" and token.valor == ")":
                break
            
            if token.tipo == "IDENTIFIER":
                parametros.append(token.valor)
                self.avanzar()
                token = self.obtener_token_actual()
                if token.valor == ",":
                    self.avanzar()
                
        return parametros
    
    def parse_expresion_aritmetica(self):
        """Analiza expresiones aritméticas complejas"""
        return self.parse_termino()
    
    def parse_termino(self):
        """Analiza términos en expresiones aritméticas"""
        resultado = self.parse_factor()
        
        while self.obtener_token_actual() and self.obtener_token_actual().tipo == "ARITHMETIC_OPERATOR":
            operador = self.avanzar()
            siguiente = self.parse_factor()
            resultado = {
                'tipo': 'operacion',
                'operador': operador.valor,
                'izquierda': resultado,
                'derecha': siguiente
            }
        
        return resultado
    
    def parse_factor(self):
        """Analiza factores en expresiones aritméticas"""
        token = self.obtener_token_actual()
        
        if token.tipo == "NUMBER":
            self.avanzar()
            return {'tipo': 'numero', 'valor': float(token.valor)}
        
        elif token.tipo == "IDENTIFIER":
            self.avanzar()
            if self.obtener_token_actual() and self.obtener_token_actual().valor == "(":
                self.avanzar()
                argumentos = self.parse_argumentos()
                self.coincidir("DELIMITER")  # )
                return {
                    'tipo': 'llamada_funcion',
                    'nombre': token.valor,
                    'argumentos': argumentos
                }
            return {'tipo': 'variable', 'nombre': token.valor}
        
        elif token.valor == "(":
            self.avanzar()
            expr = self.parse_expresion_aritmetica()
            self.coincidir("DELIMITER")  # )
            return expr
        
        raise SyntaxError(f'Token inesperado: {token}')
    
    def parse_argumentos(self):
        """Analiza los argumentos en una llamada a función"""
        argumentos = []
        while True:
            token = self.obtener_token_actual()
            if token.tipo == "DELIMITER" and token.valor == ")":
                break
                
            argumentos.append(self.parse_expresion_aritmetica())
            token = self.obtener_token_actual()
            if token.valor == ",":
                self.avanzar()
                
        return argumentos

def test_analizador():
    codigo_prueba = """
    // Definición de función para cálculo de factorial
    function factorial(n) {
        if (n <= 1) {
            return 1;
        }
        return n * factorial(n - 1);
    }

    // Función con operaciones aritméticas avanzadas
    function operaciones_avanzadas(x, y) {
        resultado = (x ^ 2) + (y // 2);  // Potencia y división entera
        modulo = x % y;                  // Operación módulo
        return resultado + modulo;
    }

    // Uso de las funciones
    resultado1 = factorial(5);
    resultado2 = operaciones_avanzadas(10, 3);
    print(resultado1);
    print(resultado2);
    """
    
    print("=== Prueba de Análisis Léxico ===")
    tokens = identificar_tokens(codigo_prueba)
    for token in tokens:
        print(f'Token: {token.tipo:20} Valor: {token.valor:10} Posición: {token.posicion}')
    
    print("\n=== Prueba de Análisis Sintáctico ===")
    parser = Parser(tokens)
    try:
        while parser.obtener_token_actual():
            parser.parse_funcion()
        print("Análisis sintáctico completado con éxito")
    except SyntaxError as e:
        print(f"Error de sintaxis: {e}")

if __name__ == "__main__":
    test_analizador()

In [ ]:
# Probando el analizador
codigo_fuente = """
function suma(a, b) {
    return a + b;
}

x = 5 ^ 2;  // Potencia
y = 10 % 3; // Módulo
resultado = suma(x, y);
print(resultado);
"""

tokens = identificar_tokens(codigo_fuente)
print("Tokens encontrados:")
for token in tokens:
    print(f'{token.tipo}: {token.valor}')